In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from torch.distributions import MultivariateNormal

In [ ]:
class Policy(torch.nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc_mean = torch.nn.Linear(dim, dim)
        self.log_std = torch.nn.Parameter(torch.zeros(dim))

    def forward(self, x):
        mean = self.fc_mean(x)
        cov = torch.diag(torch.exp(self.log_std))
        return MultivariateNormal(mean, cov)

def train(A, lambda_, policy, epochs=500, true_eigenvector=None):
    #print(true_eigenvector)
    optimizer = optim.Adam(policy.parameters())
    v = torch.randn(A.shape[0])
    v = v / torch.norm(v)

    for epoch in range(epochs):
        dist = policy(v)
        delta_v = dist.sample()

        with torch.no_grad():
            v_new = v + delta_v
            v_new = v_new / torch.norm(v_new)
            residual = torch.linalg.norm((A - lambda_ * torch.eye(A.shape[0])) @ v_new)
            reward = -residual.item()

        loss = -dist.log_prob(delta_v).sum() * reward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        v = v_new
        if epoch % 100 == 1:
            if true_eigenvector is not None:
                cosine_sim = torch.nn.functional.cosine_similarity(v.unsqueeze(0), true_eigenvector.unsqueeze(0)).item()
                print(f'Epoch {epoch}: Loss = {loss.item()}, Residual = {residual.item()}, similarity = {cosine_sim:.4f}')
            else:
                print(f'Epoch {epoch}: Loss = {loss.item()}, Residual = {residual.item()}, ')

    return v

def evaluate_model_for_matrix(A, eigenvalue, eigenvector, policy):
    v = train(A, eigenvalue, policy, epochs=10000, true_eigenvector=eigenvector)
    cosine_similarity = torch.nn.functional.cosine_similarity(v, eigenvector, dim=0)
    print(f'Cosine similarity between v and eigenvector: {cosine_similarity.item()}')


# Test function
def test_policy_on_random_matrix(dim=5, epochs=2000):
    # 1. Generate random symmetric matrix
    A_np = np.random.randn(dim, dim)
    A_np = A_np + A_np.T  # Make symmetric
    print(A_np)
    A = torch.tensor(A_np, dtype=torch.float32)
    
    # 2. Compute dominant eigenpair using numpy (for ground truth)
    eigenvalues, eigenvectors = np.linalg.eig(A_np)
    dominant_idx = np.argmax(np.abs(eigenvalues))
    dominant_lambda = eigenvalues[dominant_idx]
    dominant_v = torch.tensor(eigenvectors[:, dominant_idx], dtype=torch.float32)
    
    print(f"True dominant eigenvalue: {dominant_lambda}")
    print(f"True dominant eigenvector: {dominant_v.numpy()}")
    
    # 3. Predict using policy
    policy = Policy(dim)
    predicted_v = train(A, dominant_lambda, policy, epochs=epochs, true_eigenvector=dominant_v)
    
    # 4. Compare
    cosine_sim = torch.nn.functional.cosine_similarity(
        predicted_v.unsqueeze(0), dominant_v.unsqueeze(0)
    ).item()
    
    print(f"\nPredicted eigenvector: {predicted_v.detach().numpy()}")
    print(f"Cosine similarity with true eigenvector: {cosine_sim:.4f}")
    
    return cosine_sim

    


In [14]:
A = torch.tensor([[2.0, -1.0], [-1.0, 2.0]])
policy = Policy(A.shape[0])
evaluate_model_for_matrix(A, 3.0, torch.tensor([1.0, -1.0], dtype=torch.float32), policy)

Epoch 1: Loss = -2.1676974296569824, Residual = 0.6951131820678711, similarity = -0.9377
Epoch 101: Loss = -3.854736566543579, Residual = 1.8279554843902588, similarity = 0.4058
Epoch 201: Loss = -2.5110583305358887, Residual = 1.235765814781189, similarity = 0.7863
Epoch 301: Loss = -5.241060256958008, Residual = 1.860539436340332, similarity = 0.3669
Epoch 401: Loss = -3.9335777759552, Residual = 1.773626685142517, similarity = 0.4621
Epoch 501: Loss = -0.6507599353790283, Residual = 0.3232424855232239, similarity = 0.9869
Epoch 601: Loss = -2.8778092861175537, Residual = 1.0549179315567017, similarity = 0.8496
Epoch 701: Loss = -4.250231742858887, Residual = 1.4476673603057861, similarity = 0.6900
Epoch 801: Loss = -4.686344623565674, Residual = 1.9773706197738647, similarity = -0.1500
Epoch 901: Loss = -0.9153321981430054, Residual = 0.21460340917110443, similarity = 0.9942
Epoch 1001: Loss = -7.720101356506348, Residual = 1.969750165939331, similarity = 0.1733
Epoch 1101: Loss = -

In [16]:
test_policy_on_random_matrix(dim=10, epochs=20000)


[[ 1.17046618 -1.03623913  1.9471039   2.5377253  -1.16860613  2.46411029
  -0.24630127  1.73773302 -2.28464479  0.30277291]
 [-1.03623913 -1.10672533 -1.75385194  0.62804601  0.73394953 -0.82919344
   1.40667889  0.40194067  0.16469292 -2.72580143]
 [ 1.9471039  -1.75385194  0.07846013  2.44303351 -0.77141367 -0.90858571
  -0.07920387 -1.7514059   1.59126217  3.13485936]
 [ 2.5377253   0.62804601  2.44303351  1.98352022 -1.17562684  1.66605628
   0.86848697 -1.48914737 -1.10774632  0.05238709]
 [-1.16860613  0.73394953 -0.77141367 -1.17562684 -1.14973347  1.64879001
  -2.27620048  1.07917898  0.6349442   0.09358759]
 [ 2.46411029 -0.82919344 -0.90858571  1.66605628  1.64879001  0.62222658
   0.63566686 -2.40753123 -1.15521161  0.21077234]
 [-0.24630127  1.40667889 -0.07920387  0.86848697 -2.27620048  0.63566686
   1.26202731 -0.78540439  0.23256491  1.6485495 ]
 [ 1.73773302  0.40194067 -1.7514059  -1.48914737  1.07917898 -2.40753123
  -0.78540439  3.08741745  1.42659667  0.1333163 ]


-0.08149370551109314